# StressGait Analysis - Saliva & Self-Reports

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.stats import StatsPipeline
from fau_colors import cmaps, register_fausans_font

from IPython.display import Markdown

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
export_path = Path("../../exports")
plot_path = export_path.joinpath("plots")
bp.utils.file_handling.mkdirs([export_path, plot_path])

export_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True)
dataset

In [ ]:
order = ["Control", "OMC"]
hue_order = ["Control", "OMC"]

In [ ]:
cortisol_samples = dataset.cortisol#.unstack().head()
cortisol_samples

## Compute S0S1 Increase

The S0-S1 increase is computed as absolute increase (in nmol/l) and as increase relative to S0 (in percent).

In [ ]:
s0s1_inc = dataset.cortisol.reindex(["S0", "S1"], level="sample").unstack().diff(axis=1).dropna(axis=1, how="all")
s0s1_inc_percent = (s0s1_inc / dataset.cortisol.xs("S0", level="sample").values) * 100

s0s1_inc.columns = pd.MultiIndex.from_tuples([("cortisol", "inc_S0S1")], names=[None, "saliva_feature"])
s0s1_inc_percent.columns = pd.MultiIndex.from_tuples([("cortisol", "inc_S0S1_percent")], names=[None, "saliva_feature"])

s0s1_inc.head()

In [ ]:
cortisol_features = dataset.cortisol_features.unstack().join(s0s1_inc).join(s0s1_inc_percent)
cortisol_features = cortisol_features.stack(future_stack=True)
cortisol_features.unstack().head()

In [ ]:
cortisol_normalized = (dataset.cortisol.unstack() - dataset.cortisol.unstack()[[("cortisol", "S0")]].values).stack(
    future_stack=True
)
cortisol_normalized.unstack().head()

## Results

### Cortisol Trajectories

#### Statistics

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "mixed_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "subject": "subject",
        "within": "sample",
        "between": "condition",
        "parametric": False,
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(dataset.cortisol)

pipeline.display_results()

#### Absolute

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=dataset.cortisol,
    saliva_type="cortisol",
    sample_times=[0, 1, 1.5, 2, 4, 5],
    test_times=[1, 2],
    test_title="Gait Tests",
    sample_times_absolute=True,
    x_offset=0,
    ax=ax,
)
ax.set_ylim([2, 6])
ax.set_xticklabels([f"S{i}" for i in range(0, 6)])
ax.set_xlabel("Sample ID")

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response_absolute_sample_ids.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=dataset.cortisol,
    saliva_type="cortisol",
    sample_times=dataset.sample_times,
    test_times=[30, 38],
    test_title="Gait Tests",
    sample_times_absolute=True,
    x_offset=0,
    ax=ax,
)
ax.set_ylim([2, 6])

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response_absolute_times.pdf"), transparent=True)

#### Increase to S0

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=cortisol_normalized,
    saliva_type="cortisol",
    sample_times=[0, 1, 1.5, 2, 4, 5],
    test_times=[1, 2],
    test_title="Gait Tests",
    sample_times_absolute=True,
    x_offset=0,
    ax=ax,
)
ax.set_ylim([-1.1, 1.75])
ax.set_xticklabels([f"S{i}" for i in range(0, 6)])
ax.set_xlabel("Sample ID")
ax.set_ylabel("Cortisol Increase to S0 [nmol/l]")

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response_normalized_sample_ids.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=cortisol_normalized,
    saliva_type="cortisol",
    sample_times=dataset.sample_times,
    test_times=[30, 38],
    test_title="Gait Tests",
    sample_times_absolute=True,
    x_offset=0,
    ax=ax,
)

ax.set_ylim([-1.1, 1.75])
ax.set_ylabel("Cortisol Increase to S0 [nmol/l]")

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response_normalized_times.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=dataset.cortisol,
    saliva_type="cortisol",
    sample_times=[0, 1, 1.5, 2, 4, 5],
    test_times=[1, 2],
    test_title="Gait Tests",
    sample_times_absolute=True,
    x_offset=0,
    ax=ax,
)
ax.set_ylim([2, 6])
ax.set_xticklabels([f"S{i}" for i in range(0, 6)])
ax.set_xlabel("Sample ID")

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cortisol_response_absolute_sample_ids.pdf"), transparent=True)

#### Boxplot of Individual Cortisol Samples

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(data=dataset.cortisol.reset_index(), x="sample", y="cortisol", hue="condition", hue_order=hue_order, ax=ax)
sns.swarmplot(
    data=dataset.cortisol.reset_index(),
    x="sample",
    y="cortisol",
    hue="condition",
    hue_order=hue_order,
    ax=ax,
    palette=cmaps.faculties[:2],
    dodge=True,
    legend=False,
)

ax.set_ylabel("Cortisol [nmol/l]")

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_boxplot_cortisol_samples.pdf"), transparent=True)

#### Individual Cortisol Trajectories

In [ ]:
fig, axs = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)

dataset.cortisol.unstack()["cortisol"].xs("Control", level="condition").T.plot(ax=axs[0], legend=False, title="Control")
dataset.cortisol.unstack()["cortisol"].xs("OMC", level="condition").T.plot(ax=axs[1], legend=False, title="OMC")

axs[0].set_ylabel("Cortisol [nmol/l]")

# sns.lineplot(data=dataset.cortisol.reset_index(), x="sample", y="cortisol", ax=ax)

fig.tight_layout()

### Cortisol Features

#### Mean and Standard Deviation per Condition

In [ ]:
cortisol_features_agg = cortisol_features.groupby(["condition", "saliva_feature"]).agg(["mean", "std"]).unstack("condition")
cortisol_features_agg = cortisol_features_agg.reorder_levels([0, 2, 1], axis=1).sort_index(axis=1)
cortisol_features_agg

#### Statistical Analyses

In [ ]:
cortisol_features_to_analyze = ["inc_S0S1", "inc_S0S1_percent", "max_inc_percent", "auc_i"]

In [ ]:
cortisol_features_analysis = cortisol_features.reindex(cortisol_features_to_analyze, level="saliva_feature")
cortisol_features_analysis.unstack().head()

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "pairwise_tests")],
        params={
            "dv": "cortisol",
            "subject": "subject",
            "between": "condition",
            "parametric": False,
            "multicomp": {"method": "bonf", "levels": True},
            "groupby": "saliva_feature"
        },
    )

pipeline.apply(cortisol_features_analysis)
pipeline.display_results()

#### Plots

In [ ]:
fig, axs = plt.subplots(ncols=4)

for key, ax in zip(["inc_S0S1", "inc_S0S1_percent", "max_inc_percent", "auc_i"], axs, strict=False):
    data_slice = cortisol_features.xs(key, level="saliva_feature")
    sns.boxplot(
        data=data_slice.reset_index(), x="condition", y="cortisol", hue="condition", order=order, ax=ax, showmeans=True
    )

    ax.set_title(key)

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_boxplot_cortisol_features.pdf"), transparent=True)

### Analysis Responder vs. Non-Responder

#### Considering S0S1 increase

In [ ]:
responder_list = cortisol_features.xs("inc_S0S1", level="saliva_feature") > 1.5

# display(responder_list)

pd.DataFrame(responder_list["cortisol"].groupby("condition").value_counts())

#### Considering Maximum increase

In [ ]:
responder_list = cortisol_features.xs("max_inc_percent", level="saliva_feature") > 1.5

# display(responder_list)

pd.DataFrame(responder_list["cortisol"].groupby("condition").value_counts())